In [0]:
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

In [2]:
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

     |████████████████████████████████| 993kB 5.0MB/s 


In [0]:
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [0]:
link = 'https://drive.google.com/open?id=1ZjnmJQOGHkYD5dEAWO_vIwjZ5jEWxLb4'

In [5]:
fluff, id = link.split('=')
print(id)

1ZjnmJQOGHkYD5dEAWO_vIwjZ5jEWxLb4


In [0]:
downloaded = drive.CreateFile({'id':id})
downloaded.GetContentFile('UCI_Credit_Card.csv')
df = pd.read_csv('UCI_Credit_Card.csv')

In [7]:
df.head()

,ID,LIMIT_BAL,SEX,EDUCATION,MARRIAGE,AGE,PAY_0,PAY_2,PAY_3,PAY_4,PAY_5,PAY_6,BILL_AMT1,BILL_AMT2,BILL_AMT3,BILL_AMT4,BILL_AMT5,BILL_AMT6,PAY_AMT1,PAY_AMT2,PAY_AMT3,PAY_AMT4,PAY_AMT5,PAY_AMT6,default.payment.next.month
0,1,20000.0,2,2,1,24,2,2,-1,-1,-2,-2,3913.0,3102.0,689.0,0.0,0.0,0.0,0.0,689.0,0.0,0.0,0.0,0.0,1
1,2,120000.0,2,2,2,26,-1,2,0,0,0,2,2682.0,1725.0,2682.0,3272.0,3455.0,3261.0,0.0,1000.0,1000.0,1000.0,0.0,2000.0,1
2,3,90000.0,2,2,2,34,0,0,0,0,0,0,29239.0,14027.0,13559.0,14331.0,14948.0,15549.0,1518.0,1500.0,1000.0,1000.0,1000.0,5000.0,0
3,4,50000.0,2,2,1,37,0,0,0,0,0,0,46990.0,48233.0,49291.0,28314.0,28959.0,29547.0,2000.0,2019.0,1200.0,1100.0,1069.0,1000.0,0
4,5,50000.0,1,2,1,57,-1,0,-1,0,0,0,8617.0,5670.0,35835.0,20940.0,19146.0,19131.0,2000.0,36681.0,10000.0,9000.0,689.0,679.0,0


In [8]:
df.isnull().values.any()

False

In [9]:
df = df.rename(columns = {'default.payment.next.month':'default'})
df['default'].value_counts()

0    23364
1     6636
Name: default, dtype: int64

In [10]:
df['EDUCATION'].value_counts()

2    14030
1    10585
3     4917
5      280
4      123
6       51
0       14
Name: EDUCATION, dtype: int64

In [11]:
df['MARRIAGE'].value_counts()

2    15964
1    13659
3      323
0       54
Name: MARRIAGE, dtype: int64

In [0]:
df.query('EDUCATION!=5 and EDUCATION!=6 and EDUCATION!=0 and MARRIAGE!=0',inplace = True)

In [13]:
df['EDUCATION'].value_counts()

2    14024
1    10581
3     4873
4      123
Name: EDUCATION, dtype: int64

In [14]:
df = df.reset_index(drop = True)
df.head()

,ID,LIMIT_BAL,SEX,EDUCATION,MARRIAGE,AGE,PAY_0,PAY_2,PAY_3,PAY_4,PAY_5,PAY_6,BILL_AMT1,BILL_AMT2,BILL_AMT3,BILL_AMT4,BILL_AMT5,BILL_AMT6,PAY_AMT1,PAY_AMT2,PAY_AMT3,PAY_AMT4,PAY_AMT5,PAY_AMT6,default
0,1,20000.0,2,2,1,24,2,2,-1,-1,-2,-2,3913.0,3102.0,689.0,0.0,0.0,0.0,0.0,689.0,0.0,0.0,0.0,0.0,1
1,2,120000.0,2,2,2,26,-1,2,0,0,0,2,2682.0,1725.0,2682.0,3272.0,3455.0,3261.0,0.0,1000.0,1000.0,1000.0,0.0,2000.0,1
2,3,90000.0,2,2,2,34,0,0,0,0,0,0,29239.0,14027.0,13559.0,14331.0,14948.0,15549.0,1518.0,1500.0,1000.0,1000.0,1000.0,5000.0,0
3,4,50000.0,2,2,1,37,0,0,0,0,0,0,46990.0,48233.0,49291.0,28314.0,28959.0,29547.0,2000.0,2019.0,1200.0,1100.0,1069.0,1000.0,0
4,5,50000.0,1,2,1,57,-1,0,-1,0,0,0,8617.0,5670.0,35835.0,20940.0,19146.0,19131.0,2000.0,36681.0,10000.0,9000.0,689.0,679.0,0


**EDA**

In [15]:
df.describe()

,ID,LIMIT_BAL,SEX,EDUCATION,MARRIAGE,AGE,PAY_0,PAY_2,PAY_3,PAY_4,PAY_5,PAY_6,BILL_AMT1,BILL_AMT2,BILL_AMT3,BILL_AMT4,BILL_AMT5,BILL_AMT6,PAY_AMT1,PAY_AMT2,PAY_AMT3,PAY_AMT4,PAY_AMT5,PAY_AMT6,default
count,29601.000000,29601.000000,29601.000000,29601.000000,29601.000000,29601.000000,29601.000000,29601.000000,29601.000000,29601.000000,29601.000000,29601.000000,29601.000000,29601.000000,2.960100e+04,29601.000000,29601.000000,29601.000000,29601.000000,2.960100e+04,29601.000000,29601.000000,29601.000000,29601.000000,29601.000000
mean,14971.758927,167550.544914,1.603189,1.815479,1.555454,35.464072,-0.014932,-0.131313,-0.163440,-0.218303,-0.263978,-0.287558,50957.432012,48942.189554,4.680320e+04,43122.554204,40235.545184,38858.449816,5649.560319,5.894788e+03,5198.415898,4828.659268,4795.032735,5181.326374,0.223134
std,8660.184430,129944.020953,0.489244,0.710399,0.518092,9.213243,1.124503,1.199642,1.199793,1.172220,1.136217,1.152206,73370.242404,70923.985151,6.912389e+04,64196.383913,60699.344884,59519.893043,16568.264941,2.308919e+04,17580.914806,15711.057992,15244.217154,17657.260739,0.416355
min,1.000000,10000.000000,1.000000,1.000000,1.000000,21.000000,-2.000000,-2.000000,-2.000000,-2.000000,-2.000000,-2.000000,-165580.000000,-69777.000000,-1.572640e+05,-170000.000000,-81334.000000,-339603.000000,0.000000,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000
25%,7474.000000,50000.000000,1.000000,1.000000,1.000000,28.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,3528.000000,2970.000000,2.652000e+03,2329.000000,1780.000000,1278.000000,1000.000000,8.250000e+02,390.000000,298.000000,259.000000,138.000000,0.000000
50%,14953.000000,140000.000000,2.000000,2.000000,2.000000,34.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,22259.000000,21050.000000,2.003500e+04,19005.000000,18091.000000,17118.000000,2100.000000,2.007000e+03,1800.000000,1500.000000,1500.000000,1500.000000,0.000000
75%,22463.000000,240000.000000,2.000000,2.000000,2.000000,41.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,66623.000000,63497.000000,5.983000e+04,54271.000000,50072.000000,49121.000000,5005.000000,5.000000e+03,4500.000000,4014.000000,4042.000000,4000.000000,0.000000
max,30000.000000,1000000.000000,2.000000,4.000000,3.000000,79.000000,8.000000,8.000000,8.000000,8.000000,8.000000,8.000000,964511.000000,983931.000000,1.664089e+06,891586.000000,927171.000000,961664.000000,873552.000000,1.684259e+06,896040.000000,621000.000000,426529.000000,528666.000000,1.000000


In [16]:
pd.crosstab(df.SEX,df.EDUCATION)

EDUCATION,1,2,3,4
SEX,,,,
1,4353,5373,1978,42
2,6228,8651,2895,81


# **ONE HOT ENCODING**

In [0]:
from sklearn.preprocessing import OneHotEncoder
ohe = OneHotEncoder(sparse = False)
encode = ohe.fit_transform(df[['SEX','MARRIAGE','EDUCATION']])

In [0]:
df1 = pd.DataFrame(encode,columns=['Male','Female','Married','Single','OtherMarriage','GraduateSchool','University','HighSchool','OtherSchool'])
df1 = df1.reset_index(drop = True)

In [0]:
df = df.drop(['SEX','EDUCATION','MARRIAGE'],1)

In [0]:
data = pd.concat([df,df1],axis=1)

## **SCALING**

In [0]:
cols = data.columns
from sklearn.preprocessing import MinMaxScaler
mms = MinMaxScaler()
scaled = mms.fit_transform(data)
data = pd.DataFrame(scaled,columns=cols)

# **Train-Test Split**

In [0]:
X = data.drop(['ID','default'],1)
y = data['default']

In [0]:
from sklearn.model_selection import train_test_split as tts
X_train,X_test,y_train,y_test = tts(X,y,test_size=0.3,random_state=42)

In [0]:
from sklearn.model_selection import GridSearchCV, cross_val_score
from sklearn.metrics import confusion_matrix, precision_score, recall_score
from datetime import datetime

In [0]:
from sklearn.linear_model import LogisticRegression
log_reg = LogisticRegression(class_weight='balanced')
crossval_log = cross_val_score(log_reg,X_train,y_train,cv=10).mean()

In [0]:
log_reg.fit(X_train,y_train)
y_pred_log= log_reg.predict(X_test)

log_matrix = confusion_matrix(y_test, y_pred_log)
pre_log = precision_score(y_test, y_pred_log)
recall_log = recall_score(y_test, y_pred_log)
time_log = str(datetime.now())

In [0]:
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier()

crossval_rf = cross_val_score(rf,X_train,y_train,cv=10).mean()

In [0]:
rf.fit(X_train,y_train)
y_pred_rf= rf.predict(X_test)

rf_matrix = confusion_matrix(y_test, y_pred_rf)
pre_rf = precision_score(y_test, y_pred_rf)
recall_rf = recall_score(y_test, y_pred_rf)
time_rf = str(datetime.now())

In [0]:
from sklearn.neighbors import KNeighborsClassifier
knn  = KNeighborsClassifier()

crossval_knn = cross_val_score(knn,X_train,y_train,cv=10).mean()

In [0]:
knn.fit(X_train,y_train)
y_pred_knn = knn.predict(X_test)

knn_matrix = confusion_matrix(y_test, y_pred_knn)
pre_knn = precision_score(y_test, y_pred_knn)
recall_knn = recall_score(y_test, y_pred_knn)
time_knn = str(datetime.now())

In [0]:
from sklearn.svm import SVC
svc  = SVC()

crossval_svc = cross_val_score(svc,X_train,y_train,cv=10).mean()

In [0]:
svc.fit(X_train,y_train)
y_pred_svc = knn.predict(X_test)
svc_matrix = confusion_matrix(y_test, y_pred_svc)
pre_svc = precision_score(y_test, y_pred_svc)
recall_svc = recall_score(y_test, y_pred_svc)
time_svc = str(datetime.now())

In [36]:
Model = ["Logistic Regression","Random Forest Classifier", "K-NN Classifier", "Support Vector Classifier"]
cross_val= [crossval_log,crossval_rf,crossval_knn,crossval_svc]
matrix = [log_matrix,rf_matrix,knn_matrix,svc_matrix]
precision = [pre_log,pre_rf,pre_knn,pre_svc]
recall = [recall_log,recall_rf,recall_knn,recall_svc]
time = [time_log,time_rf,time_knn,time_svc]
results=pd.DataFrame({'TimeStamp':time,'Model': Model,'CrossValidation Score': cross_val,'Accuracy Matrix': matrix,'Precision': precision,'Recall': recall})
results


,TimeStamp,Model,CrossValidation Score,Accuracy Matrix,Precision,Recall
0,2019-09-06 14:12:14.180290,Logistic Regression,0.697251,"[[4936, 1923], [741, 1281]]",0.399813,0.633531
1,2019-09-06 14:12:51.197047,Random Forest Classifier,0.807194,"[[6441, 418], [1422, 600]]",0.589391,0.296736
2,2019-09-06 14:13:30.691967,K-NN Classifier,0.794305,"[[6313, 546], [1331, 691]]",0.558610,0.341741
3,2019-09-06 14:17:22.808731,Support Vector Classifier,0.784074,"[[6313, 546], [1331, 691]]",0.558610,0.341741
